Schema-unification & dataset merging

Building a unified schema for merging all existing datasets for furter EDA & feature engineering.
- It maps each dataset to a common schema, fills string nulls with "N/A", keeps numerics as real NaNs, and saves a single merged file.

What this does:
- Reads your already-deduped CSVs from prepared/ (no extra file finding).
- Maps each dataset to the unified columns (no country, no source).
- Trims and normalizes text, fills string nulls with "N/A"; numeric salary_numeric stays NaN.
- Concatenates all three into jobs_merged_unified.csv under prepared/.

In [9]:
import numpy as np
import pandas as pd
from pathlib import Path
%run ./hidden.ipynb

# ----------------------------
# Paths (all from prepared/)
# ----------------------------
p_mr = datasets_folder_path / "prepared" / "morocco_deduped.csv"
p_dc = datasets_folder_path / "prepared" / "dice_com_deduped.csv"
p_sl = datasets_folder_path / "prepared" / "srilanka_deduped.csv"

# ----------------------------
# Unified schema (no country, no source)
# ----------------------------
UNIFIED_COLS = [
    "job_title",
    "role",
    "company",
    "location",
    "employment_type",
    "skills",
    "job_description",
    "responsibilities",
    "qualifications",
    "experience",
    "salary",          # textual range if present
    "salary_numeric",  # numeric if present
    "post_date"
]

def _strip_strings(df: pd.DataFrame) -> pd.DataFrame:
    """Trim whitespace in all object columns."""
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip()
        # Normalize empty/placeholder to NaN (so we can fill consistently later)
        df.loc[df[c].isin(["", "nan", "None", "N/A", "NA", "NaN"]), c] = pd.NA
    return df

def _finalize_strings(df: pd.DataFrame) -> pd.DataFrame:
    """Fill missing string columns with 'N/A' (leave numerics as NaN)."""
    obj_cols = df.select_dtypes(include="object").columns.tolist()
    df[obj_cols] = df[obj_cols].fillna("N/A")
    return df

# ----------------------------
# Morocco mapping → unified
# ----------------------------
def load_morocco_unified(path: str | Path): 
    df = pd.read_csv(path, low_memory=False)
    # Expected columns (already deduped and pruned earlier)
    # ['Experience','Qualifications','Salary Range','location','Work Type',
    #  'Job Title','Role','Job Description','skills','Responsibilities',
    #  'Company','Salary_Numeric']
    out = pd.DataFrame({
        "job_title":       df.get("Job Title"),
        "role":            df.get("Role"),
        "company":         df.get("Company"),
        "location":        df.get("location"),
        "employment_type": df.get("Work Type"),
        "skills":          df.get("skills"),
        "job_description": df.get("Job Description"),
        "responsibilities":df.get("Responsibilities"),
        "qualifications":  df.get("Qualifications"),
        "experience":      df.get("Experience"),
        "salary":          df.get("Salary Range"),
        "salary_numeric":  pd.to_numeric(df.get("Salary_Numeric"), errors="coerce"),
        "post_date":       pd.NA,  # not useful/available
    })
    out = _strip_strings(out)
    # Ensure correct column order
    return out.reindex(columns=UNIFIED_COLS)

# ----------------------------
# Dice.com mapping → unified
# ----------------------------
def load_dice_unified(path: str | Path):
    df = pd.read_csv(path, low_memory=False)
    # Expected columns:
    # ['company','employmenttype_jobstatus','jobdescription',
    #  'joblocation_address','jobtitle','postdate','skills']
    out = pd.DataFrame({
        "job_title":       df.get("jobtitle"),
        "role":            pd.NA,  # not present
        "company":         df.get("company"),
        "location":        df.get("joblocation_address"),
        "employment_type": df.get("employmenttype_jobstatus"),
        "skills":          df.get("skills"),
        "job_description": df.get("jobdescription"),
        "responsibilities":pd.NA,
        "qualifications":  pd.NA,
        "experience":      pd.NA,
        "salary":          pd.NA,
        "salary_numeric":  pd.Series([np.nan] * len(df), dtype="float64"),  # Use np.nan instead of pd.NA
        "post_date":       df.get("postdate"),
    })
    out = _strip_strings(out)
    return out.reindex(columns=UNIFIED_COLS)

# ----------------------------
# Sri Lanka mapping → unified
# ----------------------------
def load_srilanka_unified(path: str | Path):
    df = pd.read_csv(path, low_memory=False)
    # Expected columns:
    # ['Job Role','Job Title','Student Qualification','Student Skills','Job Description']
    out = pd.DataFrame({
        "job_title":       df.get("Job Title"),
        "role":            df.get("Job Role"),
        "company":         pd.NA,
        "location":        pd.NA,  # country dropped; location not present
        "employment_type": pd.NA,
        "skills":          df.get("Student Skills"),
        "job_description": df.get("Job Description"),
        "responsibilities":pd.NA,
        "qualifications":  df.get("Student Qualification"),
        "experience":      pd.NA,
        "salary":          pd.NA,
        "salary_numeric":  pd.Series(np.nan, index=df.index, dtype="float64"),
        "post_date":       pd.NA,
    })
    out = _strip_strings(out)
    return out.reindex(columns=UNIFIED_COLS)

# ----------------------------
# Load each → concat → finalize
# ----------------------------
df_mr = load_morocco_unified(p_mr)
df_dc = load_dice_unified(p_dc)
df_sl = load_srilanka_unified(p_sl)

# Merge
df_all = pd.concat([df_mr, df_dc, df_sl], ignore_index=True)

# Fill strings with "N/A" (numeric stays NaN)
df_all = _finalize_strings(df_all)

# Optional: collapse multiple internal spaces & standardize commas in skills/location
def _normalize_commas(s):
    if pd.isna(s): return s
    s = " ".join(str(s).split())        # collapse whitespace
    s = s.replace(" ,", ",").replace(", ", ", ").replace(" , ", ", ")
    return s

for col in ["skills", "location"]:
    if col in df_all.columns:
        df_all[col] = df_all[col].map(_normalize_commas)

print("Unified shape:", df_all.shape)
print("Columns:", df_all.columns.tolist())
print("\nRow counts by source chunk (for reference):")
print("Morocco:", len(df_mr), "| Dice:", len(df_dc), "| SriLanka:", len(df_sl))

# ----------------------------
# Save unified merged dataset
# ----------------------------
out_path = datasets_folder_path / "processed" / "jobs_merged_unified.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
df_all.to_csv(out_path, index=False)
print(f"\nSaved merged dataset → processed /jobs_merged_unified.csv")

Unified shape: (40774, 13)
Columns: ['job_title', 'role', 'company', 'location', 'employment_type', 'skills', 'job_description', 'responsibilities', 'qualifications', 'experience', 'salary', 'salary_numeric', 'post_date']

Row counts by source chunk (for reference):
Morocco: 345 | Dice: 20598 | SriLanka: 19831

Saved merged dataset → processed /jobs_merged_unified.csv


📊 Post-Merge Quick Summary

In [10]:
# --- Quick post-merge summary ---

print("=== Unified Dataset Summary ===")
print("Shape:", df_all.shape)

# % of N/A per column
na_pct = (df_all == "N/A").mean().round(3) * 100
print("\n% of 'N/A' values per column:")
print(na_pct.sort_values(ascending=False).to_string())

# Top 10 job titles
print("\nTop 10 Job Titles:")
print(df_all['job_title'].value_counts().head(10).to_string())

# Top 10 employment types
if "employment_type" in df_all.columns:
    print("\nEmployment Type Distribution:")
    print(df_all['employment_type'].value_counts().head(10).to_string())

# Top 10 companies (non-N/A)
print("\nTop 10 Companies:")
print(df_all.loc[df_all['company'] != "N/A", 'company'].value_counts().head(10).to_string())

# Salary summary (numeric only)
if "salary_numeric" in df_all.columns:
    print("\nSalary Numeric (ignoring NaNs):")
    print(df_all['salary_numeric'].describe(percentiles=[.25,.5,.75]))

=== Unified Dataset Summary ===
Shape: (40774, 13)

% of 'N/A' values per column:
employment_type     0.5
company             0.1
skills              0.1
role                0.0
job_title           0.0
location            0.0
job_description     0.0
responsibilities    0.0
qualifications      0.0
experience          0.0
salary              0.0
salary_numeric      0.0
post_date           0.0

Top 10 Job Titles:
job_title
Cloud Infrastructure Engineer    1046
Information Security Analyst     1032
AI Engineer                      1023
System Analyst                   1008
SOC Analyst                      1001
CI/CD Engineer                    995
Help Desk Technician              982
Technical Support Engineer        980
IT Business Analyst               971
NLP Engineer                      961

Employment Type Distribution:
employment_type
<NA>                                                                                                   19831
Full Time                               